In [1]:
from torchvision import transforms

In [2]:
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from PIL import Image
import os
import torch
from sklearn.model_selection import train_test_split

    
# define a data class
class ClassificationDataset:
    def __init__(self, data, data_path, transform, training=True):
        """Define the dataset for classification problems

        Args:
            data ([dataframe]): [a dataframe that contain 2 columns: image name and label]
            data_path ([str]): [path/to/folder that contains image file]
            transform : [augmentation methods and transformation of images]
            training (bool, optional): []. Defaults to True.
        """
        self.data = data
        self.imgs = data["path"].unique().tolist()
        self.data_path = data_path
        self.training = training
        self.transform = transform

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.data_path, self.data.iloc[idx, 0]))
        if(self.training):
            label = self.data.iloc[idx, 1]
        if self.transform is not None:
            img = self.transform(img)
        if(self.training):
            return img, label
        else:
            return img

    def __len__(self):
        return len(self.imgs)


def make_loader(dataset, train_batch_size, validation_split=0.2):
    """make dataloader for pytorch training

    Args:
        dataset ([object]): [the dataset object]
        train_batch_size ([int]): [training batch size]
        validation_split (float, optional): [validation ratio]. Defaults to 0.2.

    Returns:
        [type]: [description]
    """
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
    # create train_loader
    print(len(train_set))
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=train_batch_size, shuffle=True,
    )
    # create test_loader
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False,)
    return train_loader, test_loader


def data_split(data, test_size):
    x_train, x_test, y_train, y_test = train_test_split(
        data, data["label"], test_size=test_size, stratify = data.iloc[:,1]
    )
    return x_train, x_test, y_train, y_test

In [4]:
import pandas as pd
train = pd.read_csv('/kaggle/input/world-championship-2023-embryo-classification/hvwc23/train.csv')
data_train = pd.DataFrame()
data_train['path'] = '/kaggle/input/world-championship-2023-embryo-classification/hvwc23/train/' + train['Image']
data_train['label'] = train['Class']

In [5]:
import pandas as pd
test = pd.read_csv('/kaggle/input/world-championship-2023-embryo-classification/hvwc23/test.csv')
data_test = pd.DataFrame()
data_test['path'] = '/kaggle/input/world-championship-2023-embryo-classification/hvwc23/test/' + test['Image']


# 1. Define DataLoader

In [6]:
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

In [7]:
import torchvision
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                            torchvision.transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images left-right
                            torchvision.transforms.RandomVerticalFlip(p=0.5),
                            torchvision.transforms.RandomRotation(degrees=15),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)])

In [8]:
test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)])

In [9]:
bs = 16 # batch size

In [10]:
dataset = ClassificationDataset(data_train,data_path = "",transform=transform,training=True)
train_loader,val_loader = make_loader(dataset, train_batch_size=bs, validation_split=0.2)

672


In [11]:
 X_train, X_test, y_train, y_test = train_test_split(data_train, data_train['label'],stratify=data_train['label'], test_size=0.2, random_state=42)

In [12]:
train_dataset = ClassificationDataset(X_train,data_path = "",transform=transform,training=True)
val_dataset = ClassificationDataset(X_test,data_path = "",transform=test_transform,training=True)

In [13]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
    )
    # create test_loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

In [14]:
testset = ClassificationDataset(data_test,data_path = "",transform=test_transform,training=False)
test_loader = torch.utils.data.DataLoader(
        testset, batch_size=1, shuffle=False,
    )

In [15]:
from tqdm import tqdm
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(test_loader):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

count = len(data_test) * 224 * 224

# mean and std
test_mean = psum / count
test_var  = (psum_sq / count) - (test_mean ** 2)
test_std  = torch.sqrt(test_var)

100%|██████████| 180/180 [00:01<00:00, 104.16it/s]


In [16]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs,_ in tqdm(train_loader):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

count = len(train_loader) * 224 * 224

# mean and std
train_mean = psum / count
train_var  = (psum_sq / count) - (train_mean ** 2)
train_std  = torch.sqrt(train_var)

100%|██████████| 42/42 [00:05<00:00,  7.97it/s]


In [17]:
# test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256, 256)),
#                                                torchvision.transforms.ToTensor(),
#                                                torchvision.transforms.Normalize(test_mean, test_std)])
# testset = ClassificationDataset(data_test,data_path = "",transform=test_transform,training=False)
# test_loader = torch.utils.data.DataLoader(
#         testset, batch_size=1, shuffle=False,
#     )

In [18]:
# transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256, 256)),
#                                             torchvision.transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images left-right
#                             torchvision.transforms.RandomVerticalFlip(p=0.5),
#                             torchvision.transforms.RandomRotation(degrees=15),
#                                                torchvision.transforms.ToTensor(),
#                                                torchvision.transforms.Normalize(mean, std)])
# dataset = ClassificationDataset(data_train,data_path = "",transform=transform,training=True)
# train_loader,val_loader = make_loader(dataset, train_batch_size=16, validation_split=0.2)

In [19]:
data_train

,path,label
0,/kaggle/input/world-championship-2023-embryo-c...,0
1,/kaggle/input/world-championship-2023-embryo-c...,0
2,/kaggle/input/world-championship-2023-embryo-c...,0
3,/kaggle/input/world-championship-2023-embryo-c...,0
4,/kaggle/input/world-championship-2023-embryo-c...,0
...,...,...
835,/kaggle/input/world-championship-2023-embryo-c...,1
836,/kaggle/input/world-championship-2023-embryo-c...,0
837,/kaggle/input/world-championship-2023-embryo-c...,0
838,/kaggle/input/world-championship-2023-embryo-c...,1


In [20]:
from sklearn import metrics as skmetrics
import numpy
class Metrics:
    def __init__(self, metric_names):
        self.metric_names = metric_names
        # initialize a metric dictionary
        self.metric_dict = {metric_name: [0] for metric_name in self.metric_names}

    def step(self, labels, preds):
        for metric in self.metric_names:
            # get the metric function
            do_metric = getattr(
                skmetrics, metric, "The metric {} is not implemented".format(metric)
            )
            # check if metric require average method, if yes set to 'micro' or 'macro' or 'None'
            try:
                self.metric_dict[metric].append(
                    do_metric(labels, preds, average="macro")
                )
            except:
                self.metric_dict[metric].append(do_metric(labels, preds))

    def epoch(self):
        # calculate metrics for an entire epoch
        avg = [sum(metric) / (len(metric) - 1) for metric in self.metric_dict.values()]
        metric_as_dict = dict(zip(self.metric_names, avg))
        return metric_as_dict

    def last_step_metrics(self):
        # return metrics of last steps
        values = [self.metric_dict[metric][-1] for metric in self.metric_names]
        metric_as_dict = dict(zip(self.metric_names, values))
        return metric_as_dict

In [21]:
train_metrics = Metrics(["accuracy_score","f1_score"])
val_metrics = Metrics(["accuracy_score","f1_score"])

In [22]:
!pip install focal_loss_torch


In [23]:
import torch.nn.functional as F
import torch.nn as nn

class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing
        self.dim = dim

    def forward(self, pred, target):
        target = F.one_hot(target, num_classes=pred.size(-1))
        target = target.float()
        target = (1 - self.smoothing) * target + self.smoothing / pred.size(-1)
        log_pred = F.log_softmax(pred, dim=self.dim)
        loss = nn.KLDivLoss(reduction='batchmean')(log_pred, target)
        return loss
    
criterion = LabelSmoothingLoss(smoothing=0.12)


In [24]:
import torch.nn.functional as F


class FocalLossWithSmoothing(nn.Module):
    def __init__(
            self,
            num_classes: int,
            gamma: int = 1,
            lb_smooth: float = 0.1,
            size_average: bool = True,
            ignore_index: int = None,
            alpha: float = None):
        """
        :param gamma:
        :param lb_smooth:
        :param ignore_index:
        :param size_average:
        :param alpha:
        """
        super(FocalLossWithSmoothing, self).__init__()
        self._num_classes = num_classes
        self._gamma = gamma
        self._lb_smooth = lb_smooth
        self._size_average = size_average
        self._ignore_index = ignore_index
        self._log_softmax = nn.LogSoftmax(dim=1)
        self._alpha = alpha

        if self._num_classes <= 1:
            raise ValueError('The number of classes must be 2 or higher')
        if self._gamma < 0:
            raise ValueError('Gamma must be 0 or higher')
        if self._alpha is not None:
            if self._alpha <= 0 or self._alpha >= 1:
                raise ValueError('Alpha must be 0 <= alpha <= 1')

    def forward(self, logits, label):
        """
        :param logits: (batch_size, class, height, width)
        :param label:
        :return:
        """
        logits = logits.float()
        difficulty_level = self._estimate_difficulty_level(logits, label)

        with torch.no_grad():
            label = label.clone().detach()
            if self._ignore_index is not None:
                ignore = label.eq(self._ignore_index)
                label[ignore] = 0
            lb_pos, lb_neg = 1. - self._lb_smooth, self._lb_smooth / (self._num_classes - 1)
            lb_one_hot = torch.empty_like(logits).fill_(
                lb_neg).scatter_(1, label.unsqueeze(1), lb_pos).detach()
        logs = self._log_softmax(logits)
        loss = -torch.sum(difficulty_level * logs * lb_one_hot, dim=1)
        if self._ignore_index is not None:
            loss[ignore] = 0
        return loss.mean()

    def _estimate_difficulty_level(self, logits, label):
        """
        :param logits:
        :param label:
        :return:
        """
        one_hot_key = torch.nn.functional.one_hot(label, num_classes=self._num_classes)
        if len(one_hot_key.shape) == 4:
            one_hot_key = one_hot_key.permute(0, 3, 1, 2)
        if one_hot_key.device != logits.device:
            one_hot_key = one_hot_key.to(logits.device)
        pt = one_hot_key * F.softmax(logits)
        difficulty_level = torch.pow(1 - pt, self._gamma)
        return difficulty_level

In [25]:
# criterion = FocalLossWithSmoothing(num_classes=2,lb_smooth=0.12)

# 2. Model

In [26]:
import torchvision


In [27]:
from torchvision import models
from torch import nn
model = models.efficientnet_b5(pretrained=True).cuda()
for param in model.parameters():
    param.requires_grad = False
classifier = nn.Sequential(
    nn.Linear(in_features=model.classifier[1].in_features, out_features=256,bias=True),
    nn.Linear(in_features=256, out_features=128,bias=True),
    nn.Dropout(0.2),
    nn.Linear(in_features=128, out_features=64,bias=True),
    nn.Dropout(0.2),
    nn.Linear(in_features=64, out_features=2,bias=True),
    
)
model.classifier  = classifier

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B5_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b5_lukemelas-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b5_lukemelas-b6417697.pth
100%|██████████| 117M/117M [00:04<00:00, 30.1MB/s]


In [28]:
model = model.cuda()
a= torch.zeros((1,3,224,224))
print(a.shape)
model(a.cuda())

torch.Size([1, 3, 224, 224])


tensor([[0.1097, 0.0952]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [29]:
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "min", patience=2, factor=0.5
    )

In [30]:
device = torch.device("cuda")

In [31]:
model = model.to(device)


# training loop

In [32]:
def train_one_epoch(
    model,
    train_loader,
    test_loader,
    device,
    optimizer,
    criterion,
    train_metrics,
    val_metrics,
):

    # training-the-model
    train_loss = 0
    valid_loss = 0
    all_labels = []
    all_preds = []
    model.train()
    for data, target in train_loader:
        # move-tensors-to-GPU
        data = data.type(torch.FloatTensor).to(device)
        # target=torch.Tensor(target)
        target = target.float().to(device)
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        # get the prediction label and target label
        output = model(data)
        preds = torch.argmax(output, axis=1).cpu().detach().numpy()
        labels = target.cpu().numpy()
        # calculate-the-batch-loss
        loss = criterion(output.type(torch.FloatTensor), target.type(torch.LongTensor))
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        # calculate training metrics
        all_labels.extend(labels)
        all_preds.extend(preds)
    
    train_metrics.step(all_labels, all_preds)

    # validate-the-model
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.type(torch.FloatTensor).to(device)
            target = target.to(device)
            output = model(data)
            preds = torch.argmax(output, axis=1).tolist()
            labels = target.tolist()
            all_labels.extend(labels)
            all_preds.extend(preds)
            loss = criterion(output, target)

            # update-average-validation-loss
            valid_loss += loss.item() * data.size(0)

    val_metrics.step(all_labels, all_preds)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(test_loader.sampler)

    return (
        train_loss,
        valid_loss,
        train_metrics.last_step_metrics(),
        val_metrics.last_step_metrics(),
    )

In [33]:
from tqdm import tqdm
from datetime import datetime
time_str = str(datetime.now().strftime("%Y%m%d-%H%M"))

In [34]:
for param in model.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

model = model.to(device)
num_epoch = 20
best_val_acc = 0.0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        device,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if (loss<0.04):
        # no saving
        continue
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model,
            "./" +  "best.pt"
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

begin training process


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 / 20 
 Training loss: 0.29129293241671156 - Other training metrics: 
{'accuracy_score': 0.8556547619047619, 'f1_score': 0.7040091180303601}
 
 Validation loss : 0.24109940763030732 - Other validation metrics:
{'accuracy_score': 0.8511904761904762, 'f1_score': 0.4598070739549839}


Validation accuracy= 0.8511904761904762===> Save best epoch


  5%|▌         | 1/20 [00:19<06:04, 19.16s/it]

Epoch 2 / 20 
 Training loss: 0.17438733772862525 - Other training metrics: 
{'accuracy_score': 0.8943452380952381, 'f1_score': 0.7572637223429097}
 
 Validation loss : 0.19665855273515695 - Other validation metrics:
{'accuracy_score': 0.8809523809523809, 'f1_score': 0.7569444444444444}


Validation accuracy= 0.8809523809523809===> Save best epoch


...

Epoch 18 / 20 
 Training loss: 0.04430315745550962 - Other training metrics: 
{'accuracy_score': 0.9806547619047619, 'f1_score': 0.9619749023909325}
 
 Validation loss : 0.22330758517741092 - Other validation metrics:
{'accuracy_score': 0.875, 'f1_score': 0.7491289198606272}


Validation accuracy= 0.875===> No saving


 95%|█████████▌| 19/20 [05:22<00:17, 17.05s/it]

Epoch 19 / 20 
 Training loss: 0.03379643883644825 - Other training metrics: 
{'accuracy_score': 0.9880952380952381, 'f1_score': 0.9761075161772026}
 
 Validation loss : 0.25099524244710447 - Other validation metrics:
{'accuracy_score': 0.8690476190476191, 'f1_score': 0.7415384615384616}




100%|██████████| 20/20 [05:38<00:00, 16.94s/it]

Epoch 20 / 20 
 Training loss: 0.031761579415095706 - Other training metrics: 
{'accuracy_score': 0.9880952380952381, 'f1_score': 0.9763075784018191}
 
 Validation loss : 0.2261159933349561 - Other validation metrics:
{'accuracy_score': 0.8809523809523809, 'f1_score': 0.7481259370314843}




In [35]:
!pip install lion-pytorch
from lion_pytorch import Lion
optimizer = Lion(model.parameters(), lr=1e-4, weight_decay=1e-2)


In [36]:
import copy
test_model = torch.load("/kaggle/working/best.pt")
test_model = test_model.to(device)

In [37]:
def test_result(model, test_loader, device,name='no_tta_prob.npy'):
    # testing the model by turning model "Eval" mode
    model.eval()
    preds = []
    aprobs = []
    with torch.no_grad():
        for data in test_loader:
            # move-tensors-to-GPU
            data = data.to(device)
            # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
            output = model(data)
            prob = nn.Softmax(dim=1)
            # applying Softmax to results
            probs = prob(output)
            aprobs.append(probs.cpu())
            
            preds.extend(torch.argmax(probs, axis=1).tolist())
    aprobs = np.array(aprobs)
    np.save(name,aprobs)
    return preds

In [38]:
preds =test_result(test_model, test_loader, device)


/tmp/ipykernel_23/1276390006.py:18: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  aprobs = np.array(aprobs)
/tmp/ipykernel_23/1276390006.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  aprobs = np.array(aprobs)


In [39]:
test

,ID,Image
0,1,D3_693.jpg
1,2,D3_212.jpg
2,3,D3_724.jpg
3,4,D3_623.jpg
4,5,D3_210.jpg
...,...,...
175,176,D5_020.jpg
176,177,D5_133.jpg
177,178,D5_119.jpg
178,179,D5_322.jpg


In [40]:
submission = pd.read_csv("/kaggle/input/world-championship-2023-embryo-classification/hvwc23/sample_submission.csv")

In [41]:
submission['Class'] = preds

In [42]:
submission.to_csv("submission_new.csv",index=None)

In [43]:
submission

,ID,Class
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
175,176,1
176,177,0
177,178,1
178,179,0


In [44]:
!pip install ttach
import ttach as tta
test_model = torch.load("/kaggle/working/best.pt")

In [45]:
transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 180]),
        tta.Multiply(factors=[0.9, 1, 1.1]),        
    ]
)

tta_model = tta.ClassificationTTAWrapper(test_model, transforms)

In [46]:
print(submission['Class'].sum())

18


In [47]:
preds =test_result(tta_model, test_loader, device,name='tta_prob.npy')
submission['Class'] = preds
submission.to_csv("submission_tta.csv",index=None)
print(submission['Class'].sum())

19


/tmp/ipykernel_23/1276390006.py:18: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  aprobs = np.array(aprobs)
/tmp/ipykernel_23/1276390006.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  aprobs = np.array(aprobs)
